# Luhman 16 in 2021A?

We want to figure out what sort of flux-sampling we can expect with IGRINS over the course of 2 TESS Sectors.

In [ ]:
import astroplan

import lightkurve as lk
import glob
from lightkurve.lightcurve import LightCurve

from astroquery.mast import Observations

# %load /Users/obsidian/Desktop/defaults.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u

from tqdm import tqdm
import warnings

import sys
import os

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

We can use astroquery to download the `PATHOS` lightcurve.  astroquery caches the result so no need to re-downloaded.

In [ ]:
redownload=False

In [ ]:
if redownload:
    sector_num = '10'
    print('Querying for PATHOS Sector ' + sector_num + " Observations of Luhman 16.")
    obsTable = Observations.query_criteria(provenance_name = "PATHOS", objectname='Luhman 16',
                                           sequence_number = sector_num)

    print("Found a total of " + str(len(obsTable)) + " PATHOS targets.")
    print('Downloading data products for these observations...')
    for obs in obsTable:
        data_products = Observations.get_product_list(obs)
        Observations.download_products(data_products, '../data/TESS/')

In [ ]:
fns = glob.glob('../data/TESS/**/*.fits', recursive=True)
fn = fns[0]

In [ ]:
lc = LightCurve.read(fn, format='pathos')

Woohoo!  We have the lightcurve!

In [ ]:
%%capture
lc = lc.remove_outliers(sigma=5).normalize()

In [ ]:
missing_data = lc.flux < 0.87
too_early = lc.time.value < 1572.5
too_early2 = (lc.time.value > 1583.0) & (lc.time.value < 1586)
lc = lc[~missing_data & ~too_early & ~too_early2]

In [ ]:
n_epochs = 11
epochs = np.random.choice(lc.time.value, size=n_epochs)

In [ ]:
lc.plot(marker='o', linestyle='none')
for epoch in epochs:
    plt.axvline(epoch, color='r', linestyle='dashed')

In [ ]:
index_mask = np.in1d(lc.time.value, epochs)

In [ ]:
fluxes = lc.flux.value[index_mask]

In [ ]:
plt.hist(lc.flux, bins=np.arange(0.8, 1.1, 0.01));
for flux in fluxes:
    plt.axvline(flux, color='r', linestyle='dashed')

Neat!  But not all times are observable from the ground.  Let's use astroplan to figure out observing windows.

## Projecting ahead with Astropy

In [ ]:
from astroplan import Observer
from astroplan import FixedTarget
from astroplan import observability_table
from astropy.time import Time

In [ ]:
gemini = Observer.at_site('gemini south')

In [ ]:
target = FixedTarget.from_name('Luhman 16')

In [ ]:
time_start = Time('2021-03-08 00:00:00') #start of Semester
time_end = Time('2021-03-18 00:00:00') #end of Semester

In [ ]:
sunset_start = gemini.sun_set_time(time_start, which='nearest')
sunrse_start = gemini.sun_rise_time(time_start, which='nearest')
sunset_start.iso, sunrse_start.iso

In [ ]:
sunset_end = gemini.sun_set_time(time_end, which='nearest')
sunrse_end = gemini.sun_rise_time(time_end, which='nearest')
sunset_end.iso

In [ ]:
con1 = astroplan.AtNightConstraint().twilight_astronomical()
con2 = astroplan.AirmassConstraint(2.0)
at_night_low_airmass = [con1, con2]

In [ ]:
time_range = (sunset_start, sunrse_end)

In [ ]:
from astroplan.utils import time_grid_from_range

In [ ]:
sector_36_times = (sunrse_start + (lc.time -lc.time.min()))
#sector_37_times = (sector_37_start + (lc.time -lc.time.min()))

In [ ]:
at_night_low_airmass[0]

In [ ]:
observability_grid1 = con1(gemini, target, times=sector_36_times)
observability_grid2 = con2(gemini, target, times=sector_36_times)
observability_grid_36 = observability_grid1 & observability_grid2

In [ ]:
lc_future = lc.copy()

In [ ]:
lc_future.time = sector_36_times+0.5

In [ ]:
all_times = np.hstack([lc_future.time.value])
observability = np.hstack([observability_grid_36 & (lc_future.time.value < 2291)])
all_fluxes = np.hstack([lc_future.flux.value])

In [ ]:
observable_times = all_times[observability]

In [ ]:
n_epochs = 5
epochs = np.random.choice(observable_times,
                          size=n_epochs)

In [ ]:
ax = lc_future.plot(marker='s', label='Sector 36 (Simulated)')
ax.axvline(epochs[0], color='r', linestyle='dashed', label='Randomly scheduled Airmass $>2$ IGRINS visits')

for epoch in epochs[1:]:
    plt.axvline(epoch, color='r', linestyle='dashed')
    
plt.axvline(sunset_start.btjd+1, linestyle='solid', color='b', label='March 8')
plt.axvline(sunrse_end.btjd, linestyle='dotted', color='g', label='March 21')

plt.ylim(0.8, 1.10)
plt.xlim(2281, 2293)
plt.title('Luhman 16 AB IGRINS visits in 2021A with contemporaneous TESS Monitoring')
plt.legend()
plt.savefig('../proposal/2021A/DDT/tess_luhman16/figures/Luhman16AB_IGRINS_2021A_TESS_scheduling.pdf', bbox_inches='tight')

In [ ]:
#! xdg-open  ../

In [ ]:
index_mask = np.in1d(all_times, epochs)

In [ ]:
fluxes = all_fluxes[index_mask]

In [ ]:
plt.hist(lc.flux, bins=np.arange(0.85, 1.1, 0.01));
#plt.hist(lc.flux[observability], bins=np.arange(0.85, 1.1, 0.01));

for flux in fluxes:
    plt.axvline(flux, color='r', linestyle='dashed')
plt.xlabel('Flux')
plt.title('Simulated Luhman 16 AB flux sampling in 2021A')
plt.savefig('../proposal/2021A/DDT/tess_luhman16/figures/Luhman16AB_IGRINS_2021A_TESS_flux_sampling.pdf', dpi=300, bbox_inches='tight')

In [ ]:
plt.rcParams.update({
    "figure.facecolor":  (1.0, 1.0, 1.0, 1.0),  # red   with alpha = 30%
    "axes.facecolor":    (1.0, 1.0, 1.0, 1.0),  # green with alpha = 50%
    "savefig.facecolor": (1.0, 1.0, 1.0, 1.0),  # blue  with alpha = 20%
})

The end!